# Peer-graded Assignment: 
## Segmenting and Clustering Neighborhoods in Toronto

### part 1: 
based on information on wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M create a pandas dataframe with columns
    * PostalCode
    * Borough
    * Neighborhood
   

The wikipedia list is a list of postal codes in Canada where the first letter is M. 
Postal codes beginning with M are located within the city of Toronto in the province of Ontario.

### documentation
https://beautiful-soup-4.readthedocs.io/en/latest/#

 see https://www.youtube.com/watch?v=ng2o98k983k for use of beautifullsoup
    

In [1]:
# with open('simple.html') as html_file:
#    soup = BeautifulSoup(html_file, 'lxml')
# print(soup) ## the entire code, not indented   
# print(soup.prettify()) ## indented

# match = soup.title  ## finds the title tag (including tags)
# match = soup.title.text
# print(match)

# match = soup.div
# match = soup.find('div')   ## results the same (13:00)
# match = soup.find('div', class_='footer')  ## finds a certain (here underscore because class is python specific word

# article = soup.find('div', class_='article')
# headline = article.h2.a.text
# summary = article.p.text

# for article in soup.find_all('div', class_='article'):
#    headline = article.h2.a.text
#    summary = article.p.text


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181102054449-0000


### libraries install


In [2]:
## in anaconda prompt:  
## conda install -c conda-forge beautifulsoup4 --yes
## conda install -c conda-forge lxml --yes   # 4:33
## conda install -c conda-forge html5lib --yes   # not done

### libraries import


In [3]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np

### data scraping

see https://www.youtube.com/watch?v=ng2o98k983k for use of beautifullsoup

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup_raw = BeautifulSoup(source, 'html5')
# print(soup_raw.prettify())
print("done")

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/ibm/conda/miniconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


done


In [5]:
soup_table = soup_raw.tbody
# print(soup_table)
## parser has added columnnames into tbody while wikipediasite had a thead-tag
print("done")

done


In [6]:
## building the dataframe from the soup_table result
scraped_result = []
#result = pd.DataFrame(columns = ["PostalCode"])
scraped_result = pd.DataFrame(columns = ["PostalCode","Borough","Neighborhood"])
for soup_tr in soup_table.find_all('tr'):
    i = 1 
    ## for empty records in tablerows where there is no table data (like the table header line) a dummy result is given
    PostalCode = "A1A"
    Borough = "Not assigned"
    Neighborhood = "Not assigned"
    for  soup_td in soup_tr.find_all('td'):
        if i == 3:
            Neighborhood = str(soup_td.text)
            Neighborhood = Neighborhood.replace("\n","")
        if i == 2:
            Borough = str(soup_td.text)
            i = 3
        if i == 1:
            PostalCode = str(soup_td.text)
            i = 2
    newline = pd.DataFrame([[PostalCode,Borough,Neighborhood]], columns = ["PostalCode","Borough","Neighborhood"] )
    scraped_result = scraped_result.append(newline, ignore_index=True)
print('done')
# scraped_result
#newline

done


### cleaning data

***"Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned."***

In [7]:
## removing lines with "Not assigned" in Borough, this includes eventual dummy records
cleaning1 = scraped_result
cleaning1 = cleaning1.loc[cleaning1['Borough'] != "Not assigned"]
print('done')
# cleaning1

done


***If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.***

In [8]:
## where "not assigned" in Neighborhood copy the name of the borough
## assumption is made that this never applies for PostalCodes which cover multiple Neighborhoods, 
## hence the order of cleaning can be different than in the 'rules' of the assignment 
cleaning2 = cleaning1
## splitting the dataset in two parts, not to be adjusted, and to adjusted
cleaning2a = cleaning2.loc[cleaning2['Neighborhood'] != "Not assigned"]
cleaning2b = cleaning2.loc[cleaning2['Neighborhood'] == "Not assigned"]
cleaning2b['Neighborhood'] = cleaning2b['Borough'] 
## previous line induced the following error: 
##     "A value is trying to be set on a copy of a slice from a DataFrame.
##     Try using .loc[row_indexer,col_indexer] = value instead
##     See the caveats in the documentation: 
##     http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy"
## In this case it is a false positive: 
## see https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas#20627316

## and combining the two parts
cleaning2 = cleaning2a.append(cleaning2b)
print('done')
# cleaning2


done


/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


***More than one neighborhood can exist in one postal code area. 
For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
These two rows will be combined into one row with the neighborhoods separated with a comma (and a space)***

In [9]:
## the dataframe cleaning3 is buildup by adding lines from previous step (cleaning2)
cleaning3 = pd.DataFrame(columns = ["PostalCode", "Borough", "Neighborhood"])
for postalcode in set(cleaning2['PostalCode']):
    tempstring = ''
    no_of_neighborhoods = cleaning2[cleaning2['PostalCode'] == postalcode].shape[0]
    ## making a small dataframe with just the current postalcode
    tempdata = cleaning2[cleaning2['PostalCode'] == postalcode]
    tempdata = tempdata.reset_index()
    for i in range(0,no_of_neighborhoods):
        ## the assumption is made that each PostalCode, even when it compasses more than one Neighborhood
        ## it only compasses only one Borough. so it doesn't matter from which line the borough information comes
        borough = tempdata.loc[i,'Borough']
        strippednb = tempdata.loc[i,'Neighborhood'].strip()
        tempstring = tempstring + strippednb 
        if i < no_of_neighborhoods-1:
            tempstring = tempstring + ", "
    newline = pd.DataFrame([[postalcode,borough,tempstring]], columns = ["PostalCode","Borough","Neighborhood"] )
    cleaning3 = cleaning3.append(newline)

cleaning3 = cleaning3.reset_index()
print('done')
#cleaning3 


done


In [10]:
resultA = cleaning3

## results

In [11]:
resultA

,index,PostalCode,Borough,Neighborhood
0,0,M4J,East York,East Toronto
1,0,M5S,Downtown Toronto,"Harbord, University of Toronto"
2,0,M6S,West Toronto,"Runnymede, Swansea"
3,0,M6H,West Toronto,"Dovercourt Village, Dufferin"
4,0,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
5,0,M1J,Scarborough,Scarborough Village
6,0,M6E,York,Caledonia-Fairbanks
7,0,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."
8,0,M5G,Downtown Toronto,Central Bay Street
9,0,M5C,Downtown Toronto,St. James Town


In [12]:
resultA.shape

(103, 4)